# 1. Readme


## 1.1 Required packages

In [1]:
using JuMP ## optimization
using Ipopt ## nonlinear optimization
 
using ForwardDiff ## autodiff - gradient and hessian
using LinearAlgebra
using Random
using Statistics
using Distributions

using CSV
using DataFrames
using Pipe

using Plots
using Makie
using CairoMakie

using Latexify
using ZipFile

## 1.3 Functions for optimization

In [2]:
include("functions.jl")

sample_from_n (generic function with 1 method)

# 2. Empirical study

## 2.1 Read the data

Download data from Github repository

In [11]:
download("https://github.com/DepartmentOfStatisticsPUE/job-vacancies-capture-recapture/blob/main/data-for-capture-recapture.csv.zip?raw=true",
         "data-for-capture-recapture.csv.zip")

"data-for-capture-recapture.csv.zip"

In [3]:
z = ZipFile.Reader("../data/data-for-capture-recapture.csv.zip") 
a_file_in_zip = filter(x -> x.name == "data-for-capture-recapture.csv", z.files)[1]
cr_data = CSV.read(a_file_in_zip, DataFrame) 
first(cr_data, 10)

,date_start,date_end,date_arch,vacancies,q1,q2,q3,q4,q1_days
,Date…,Date…,Date…?,Int64,Bool,Bool,Bool,Bool,Int64?
1,2018-03-21,2018-04-03,2018-04-04,1,1,0,0,0,10
2,2018-12-03,2019-01-02,missing,1,0,0,0,1,missing
3,2018-06-15,2018-07-15,missing,1,0,1,0,0,missing
4,2018-12-31,2019-01-30,missing,1,0,0,0,1,missing
5,2018-06-15,2018-07-15,missing,1,0,1,0,0,missing
6,2018-03-22,2018-04-21,missing,1,1,0,0,0,9
7,2018-12-14,2019-01-13,missing,1,0,0,0,1,missing
8,2018-09-27,2018-10-27,missing,1,0,0,1,0,missing
9,2018-06-12,2018-07-12,missing,1,0,1,0,0,missing


In [4]:
Random.seed!(123)
results_q1 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :size, "D", true, 500, true);
results_q2 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :size, "D", true, 500, true);
results_q3 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :size, "D", true, 500, true);
results_q4 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :size, "D", true, 500, true);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Population size: 73237.4628252493
Gradient: 0.00015754839130377452
Bootstrap started
Population size: 62999.97753536585
Gradient: 0.00020073123857711206
Bootstrap started
Population size: 63426.49039699523
Gradient: 0.00010058841778737282
Bootstrap started
Population size: 48931.107224776344
Gradient: 4.512812357088114e-5
Bootstrap started


Naive estimators for comparison

In [27]:
@pipe create_data(cr_data, :q1, :q1_days, 30) |>
      groupby(_, [:cbop, :pracuj]) |>
      combine(_, :count => sum => :N) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      ( (_.N[1] + _.N[2])*(_.N[1]+_.N[3])/_.N[1]) |> println

@pipe create_data(cr_data, :q2, :q2_days, 30) |>
      groupby(_, [:cbop, :pracuj]) |>
      combine(_, :count => sum => :N) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      ( (_.N[1] + _.N[2])*(_.N[1]+_.N[3])/_.N[1]) |> println

@pipe create_data(cr_data, :q3, :q3_days, 30) |>
      groupby(_, [:cbop, :pracuj]) |>
      combine(_, :count => sum => :N) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      ( (_.N[1] + _.N[2])*(_.N[1]+_.N[3])/_.N[1]) |> println

@pipe create_data(cr_data, :q4, :q4_days, 30) |>
      groupby(_, [:cbop, :pracuj]) |>
      combine(_, :count => sum => :N) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      ( (_.N[1] + _.N[2])*(_.N[1]+_.N[3])/_.N[1]) |> println


153959.76340694007
132548.2334739803
127224.1502276176
99694.39577836411


In [28]:
@pipe create_data(cr_data, :q1, :q1_days, 30) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      groupby(_, [:size2]) |> 
      combine(_, :count => (x -> (x[1]+x[2])*(x[1]+x[3])/x[1] )) |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.f") |> println

@pipe create_data(cr_data, :q2, :q2_days, 30) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      groupby(_, [:size2]) |> 
      combine(_, :count => (x -> (x[1]+x[2])*(x[1]+x[3])/x[1] ))   |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.f")|> println

@pipe create_data(cr_data, :q3, :q3_days, 30) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      groupby(_, [:size2]) |> 
      combine(_, :count => (x -> (x[1]+x[2])*(x[1]+x[3])/x[1] ))   |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.f")|> println

@pipe create_data(cr_data, :q4, :q4_days, 30) |>
      sort(_, [:cbop, :pracuj], rev = true) |>
      groupby(_, [:size2]) |> 
      combine(_, :count => (x -> (x[1]+x[2])*(x[1]+x[3])/x[1] ))   |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.f")|> println


\begin{tabular}{rr}
size2 & count_function\\
SM & 336690\\
D & 25189\\
\end{tabular}

\begin{tabular}{rr}
size2 & count_function\\
SM & 247647\\
D & 23664\\
\end{tabular}

\begin{tabular}{rr}
size2 & count_function\\
SM & 250189\\
D & 23591\\
\end{tabular}

\begin{tabular}{rr}
size2 & count_function\\
SM & 154694\\
D & 21180\\
\end{tabular}



# 3. Reporting

## 3.1 Expected values and variances from bootstrap

In [30]:
@pipe vcat(
    mean(results_q1[3], dims = 1),
    mean(results_q2[3], dims = 1),
    mean(results_q3[3], dims = 1),
    mean(results_q4[3], dims = 1)
)  |> transpose(_) |>
    DataFrame(_, [:Q1, :Q2, :Q3, :Q4]) |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.4f")

L"\begin{tabular}{rrrr}
Q1 & Q2 & Q3 & Q4\\
54620.0501 & 46003.8364 & 45971.4452 & 33966.5128\\
18916.0023 & 17360.2336 & 17752.5753 & 15180.5158\\
0.0690 & 0.0806 & 0.0702 & 0.0757\\
0.1651 & 0.1895 & 0.1810 & 0.1212\\
0.0119 & 0.0161 & 0.0138 & 0.0200\\
0.1837 & 0.1923 & 0.1847 & 0.1790\\
\end{tabular}
"

Expected value for $N_A+N_B=N$

In [31]:
sum(mean(results_q1[3], dims = 1)[1:2]), 
sum(mean(results_q2[3], dims = 1)[1:2]),
sum(mean(results_q3[3], dims = 1)[1:2]),
sum(mean(results_q4[3], dims = 1)[1:2]) 

(73536.052424476, 63364.06994960985, 63724.020467564565, 49147.02858792442)

Standard errors based on the bootstrap

In [10]:
@pipe vcat(
    std(results_q1[3], dims = 1),
    std(results_q2[3], dims = 1),
    std(results_q3[3], dims = 1),
    std(results_q4[3], dims = 1)
)  |> transpose(_) |>
    DataFrame(_, [:Q1, :Q2, :Q3, :Q4]) |>
    latexify(_; env=:table, latex=false, adjustment = :r, fmt = "%.4f")

L"\begin{tabular}{rrrr}
Q1 & Q2 & Q3 & Q4\\
2626.2206 & 2223.5112 & 2275.3450 & 2412.8609\\
776.9516 & 706.8354 & 734.2906 & 909.3264\\
0.0053 & 0.0065 & 0.0058 & 0.0088\\
0.0077 & 0.0088 & 0.0086 & 0.0082\\
0.0012 & 0.0014 & 0.0013 & 0.0023\\
0.0079 & 0.0083 & 0.0080 & 0.0102\\
\end{tabular}
"

Standard error for $N_A + N_B = N$

In [11]:
std(sum(results_q1[3][:,1:2], dims = 2)), 
std(sum(results_q2[3][:,1:2], dims = 2)),
std(sum(results_q3[3][:,1:2], dims = 2)),
std(sum(results_q4[3][:,1:2], dims = 2))

(3358.3751164165574, 2893.4105728358804, 2968.8519812256263, 3278.6344934812037)

## 3.2 Confidence intervals

In [21]:
expected = vcat(
    mean(results_q1[3], dims = 1),
    mean(results_q2[3], dims = 1),
    mean(results_q3[3], dims = 1),
    mean(results_q4[3], dims = 1)
)

stds = vcat(
    std(results_q1[3], dims = 1),
    std(results_q2[3], dims = 1),
    std(results_q3[3], dims = 1),
    std(results_q4[3], dims = 1)
)

zscore = quantile(Normal(), 1- 0.05/2)

groupes_ci = hcat(
    expected[:,1:2] .- zscore .* stds[:,1:2],
    expected[:,1:2] .+ zscore .* stds[:,1:2]
    )[:,[1,3,2,4]]

latexify(groupes_ci; env=:table, latex=false, adjustment = :r, fmt = "%.0f")

L"\begin{tabular}{rrrr}
49473 & 59767 & 17393 & 20439\\
41646 & 50362 & 15975 & 18746\\
41512 & 50431 & 16313 & 19192\\
29237 & 38696 & 13398 & 16963\\
\end{tabular}
"

In [26]:
total_ci = hcat(
    sum(expected[:,1:2], dims = 2) .- zscore .* sum(stds[:,1:2], dims= 2),
    sum(expected[:,1:2], dims = 2) .+ zscore .* sum(stds[:,1:2], dims= 2),    
)
latexify(total_ci; env=:table, latex=false, adjustment = :r, fmt = "%.0f")

L"\begin{tabular}{rr}
66866 & 80206\\
57621 & 69107\\
57825 & 69623\\
42636 & 55658\\
\end{tabular}
"

# Appendix

## D  -- selection of stratification variables

In [73]:
## small -> M (pol. małe)
results_q1_a1 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :size, "M", false, 500, false);
results_q2_a1 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :size, "M", false, 500, false);
results_q3_a1 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :size, "M", false, 500, false);
results_q4_a1 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :size, "M", false, 500, false);

## medium -> S (pol. srednie)
results_q1_a2 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :size, "S", false, 500, false);
results_q2_a2 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :size, "S", false, 500, false);
results_q3_a2 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :size, "S", false, 500, false);
results_q4_a2 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :size, "S", false, 500, false);

## sector 
results_q1_a3 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :sector, 1, false, 500, false);
results_q2_a3 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :sector, 1, false, 500, false);
results_q3_a3 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :sector, 1, false, 500, false);
results_q4_a3 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :sector, 1, false, 500, false);

## PKD -- G (largest)
results_q1_a4 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :pkd, "G", false, 500, false);
results_q2_a4 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :pkd, "G", false, 500, false);
results_q3_a4 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :pkd, "G", false, 500, false);
results_q4_a4 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :pkd, "G", false, 500, false);

## PKD -- C (second from top 3)
results_q1_a5 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :pkd, "C", false, 500, false);
results_q2_a5 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :pkd, "C", false, 500, false);
results_q3_a5 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :pkd, "C", false, 500, false);
results_q4_a5 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :pkd, "C", false, 500, false);

## PKD -- M (highest on pracuj)
results_q1_a6 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :pkd, "M", false, 500, false);
results_q2_a6 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :pkd, "M", false, 500, false);
results_q3_a6 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :pkd, "M", false, 500, false);
results_q4_a6 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :pkd, "M", false, 500, false);

## PKD -- M (highest on pracuj)
results_q1_a7 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :pkd, "I", false, 500, false);
results_q2_a7 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :pkd, "I", false, 500, false);
results_q3_a7 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :pkd, "I", false, 500, false);
results_q4_a7 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :pkd, "I", false, 500, false);

## woj -- 14 (where the capital is located)
results_q1_a8 = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :woj, 14, false, 500, false);
results_q2_a8 = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :woj, 14, false, 500, false);
results_q3_a8 = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :woj, 14, false, 500, false);
results_q4_a8 = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :woj, 14, false, 500, false);



In [79]:
results_strata = map(x -> sum(x[1][1:2]), 
                         [results_q1_a1, results_q2_a1, results_q3_a1, results_q4_a1,
                          results_q1_a2, results_q2_a2, results_q3_a2, results_q4_a2,
                          results_q1_a3, results_q2_a3, results_q3_a3, results_q4_a3,
                          results_q1_a4, results_q2_a4, results_q3_a4, results_q4_a4,
                          results_q1_a5, results_q2_a5, results_q3_a5, results_q4_a5,
                          results_q1_a6, results_q2_a6, results_q3_a6, results_q4_a6,
                          results_q1_a7, results_q2_a7, results_q3_a7, results_q4_a7,
                          results_q1_a8, results_q2_a8, results_q3_a8, results_q4_a8])

results_strata_res = reshape(results_strata, 4, 8)'

latexify(results_strata_res; env=:table, latex=false, adjustment = :r, fmt = "%.0f")

L"\begin{tabular}{rrrr}
99605 & 81357 & 80729 & 61077\\
31141 & 31766 & 36105 & 33163\\
104583 & 124812 & 97505 & 71831\\
141674 & 126785 & 121292 & 95381\\
19845 & 19405 & 18370 & 28239\\
142199 & 122570 & 119051 & 93220\\
101602 & 125072 & 81941 & 55319\\
132167 & 111632 & 111428 & 91009\\
\end{tabular}
"

Loop over NACE levels  -- only for q1 becuase for other quarters data is to sparse

In [138]:
results_nace = Float64[]

for lev in sort(unique(cr_data.pkd))
    nace_lev = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :pkd, lev, false, 500, false);
    append!(results_nace, sum(nace_lev[1][1:2]))
end

results_nace

19-element Vector{Float64}:
 112267.1283880904
 102644.86786560943
  19844.898254177315
 153228.71670775797
  79814.68255102547
 135666.00366553615
 141673.9602137246
 134686.73749203858
 101602.15849559402
 145546.05444026727
 153462.37179220864
 152272.13559644777
 142198.98112161297
  22907.517558473708
 129977.60653657423
 149284.6095797069
 116836.82735852922
 121361.28144263117
 123765.07486925594

Loop over WOJ levels 

In [152]:
results_woj = Float64[]

for lev in sort(unique(cr_data.woj))
    woj_lev = cr_averse_analysis(cr_data, :q1, :q1_days, 30, :woj, lev, false, 500, false);
    append!(results_woj, sum(woj_lev[1][1:2]))
    woj_lev = cr_averse_analysis(cr_data, :q2, :q2_days, 30, :woj, lev, false, 500, false);
    append!(results_woj, sum(woj_lev[1][1:2]))
    woj_lev = cr_averse_analysis(cr_data, :q3, :q3_days, 30, :woj, lev, false, 500, false);
    append!(results_woj, sum(woj_lev[1][1:2]))
    woj_lev = cr_averse_analysis(cr_data, :q4, :q4_days, 30, :woj, lev, false, 500, false);
    append!(results_woj, sum(woj_lev[1][1:2]))    
end

results_woj_res = reshape(results_woj, 4, 16)'

results_woj_res ./ 1000

16×4 Matrix{Float64}:
 127.563    33.0676   61.1725  31.3906
  93.5114   72.9935   75.0192  74.004
 150.748   115.817    90.0773  85.3805
  98.1754   98.2765   83.7968  92.4203
  77.8125  121.895   107.977   70.9222
 151.918   130.824   124.369   98.9592
 132.167   111.632   111.428   91.0086
  79.4915   79.8815   75.1014  71.315
  87.6543   71.2824   87.7866  68.1609
  90.496    76.4548  108.222   82.4273
 153.253   126.138   119.465   65.3645
 122.791   120.479   114.304   92.7219
  95.0892   83.8569  108.671   65.7334
  72.8086   93.3758  101.735   43.5122
 148.395   130.869   125.79    98.6008
  96.8934  117.876    90.7329  96.4235